# Plot comparison between $M_0$ and $T_{hold}$

Plot the comparison to evaluate the time healing of the friction on the gouge patch.

2025.01.05 Kurama Okubo

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import matplotlib as mpl
from matplotlib import ticker

%matplotlib inline
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm
import warnings
import time
import pickle 
import seaborn as sns
import copy

import scipy.io as sio
from scipy.signal import decimate

%load_ext autoreload
%autoreload 2

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 12
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = False #True


In [ ]:
# datadir = "../data/M0vsThold/"
# if not os.path.exists(datadir):
#     os.makedirs(datadir)

figdir = "../figure/M0vsThold/"
if not os.path.exists(figdir):
    os.makedirs(figdir)


In [ ]:
expr_id = 87

In [ ]:
# Load macroscopic data
fi_macroscropic = "../../../Experiments/MacroData/data/MacroData_raw.mat"
fi_macro = sio.loadmat(fi_macroscropic, squeeze_me=False)

In [ ]:
# fi_macro
tvec_macro = np.squeeze(fi_macro["M"][f"FB03_{expr_id:03d}"][0][0]["tmat"][0][0])
SS = np.squeeze(fi_macro["M"][f"FB03_{expr_id:03d}"][0][0]["SS"][0][0])
NP = np.squeeze(fi_macro["M"][f"FB03_{expr_id:03d}"][0][0]["NP"][0][0])
dt_macro = tvec_macro[1] - tvec_macro[0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(7, 5))

ax1.plot(tvec_macro, SS, "k-")
ax1.set_xlim([0, 270])
ax1.set_ylim([0.6, 0.7])
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Macro Shear stress [MPa]")

ax2.plot(tvec_macro, SS, "k.-")
ax2.set_xlim([151.5, 152])
ax2.set_ylim([0.6, 0.7])
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Macro Shear stress [MPa]")

fig.tight_layout()

# Plot gouge events and the macroscopic stress

In [ ]:
# read gouge event catalog
gougepatch_id = "G3"
Qinv_quart = 50

Nvalidsensors_thresh = 4

finame = f"../../../GougeEventCatalog/data/gougeeventcatalog__fb03-{expr_id:03d}__{gougepatch_id}__Q{Qinv_quart}.csv"
df_gc = pd.read_csv(finame, index_col=0)
df_gc_selected = df_gc[df_gc["Nvalidsensors"] >= Nvalidsensors_thresh].copy()
df_gc_selected.head()

In [ ]:
len(df_gc_selected)

In [ ]:
# Decimate the SS to clean the line plot and to reduce file size
# decimate 100 times

# NOTE: signal.decimate applies an anti-aliasing filter, which changes the offset in our data set as it contains the unloading part.
# Therefore, we simply decimate the data by plot step instead of using the signal.decimate.
# The result is nealy identical and no problem in the comparison.

# tvec_macro_decimate = decimate(tvec_macro,10)
# tvec_macro_decimate = decimate(tvec_macro_decimate,10)
# tvec_macro_decimate = tvec_macro[::100]
# SS_decimate = decimate(SS,10)
# SS_decimate = decimate(SS_decimate,10)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3.6))

plotstep_SS = 100
# ax.plot(tvec_macro, SS, "r-", lw=1)
ax.plot(tvec_macro[::plotstep_SS], SS[::plotstep_SS], "k-", lw=1.2, zorder=4)
# ax.plot(tvec_macro_decimate, SS_decimate+0.015, "b-", lw=1)

ax.set_xlim([0, 270])
ax.set_ylim([0.6, 0.7])
ax.set_xlabel("Time [s]")
ax.set_ylabel("Macroscopic shear stress [MPa]")

ax2 = ax.twinx()

# Plot foreshock
df_gc_fore = df_gc_selected[df_gc_selected["event_label"]=="F"]
df_gc_after = df_gc_selected[df_gc_selected["event_label"]=="A"]

df_gc_fore.plot.scatter(x="event_onset_time", y="Mw", s=30, ax=ax2, color="r", marker="o", edgecolor="k", zorder=3, label="Foreshock")
df_gc_after.plot.scatter(x="event_onset_time", y="Mw", s=30, ax=ax2, color="b", marker="s", edgecolor="k", zorder=2, label="Aftershock")

# plot vertical bars
ylimit2 = [-8.0, -5.9]

for index, row in df_gc_selected.iterrows():
    event_t = row["event_onset_time"]
    Mw = row["Mw"]
    ax2.plot([event_t, event_t], [ylimit2[0], Mw], "-", c="gray", zorder=1, lw=0.75, alpha=0.5)

ax2.set_ylim(ylimit2)
ax2.legend(loc=4)

# plt.suptitle(f'Q{Qinv_quart} water-level={k_waterlevel:.2f}', y=0.98)
plt.tight_layout()

plt.savefig(figdir+f"/M0vsSS_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.png", format="png", dpi=200)
# plt.savefig(figdir+f"/M0vsSS_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.pdf")


# Compute $T_{hold}$

We compute the hold time of the gouge events by the duration from the end of previous stick-slip to the onset of events. For the clarity, we focus only on the foreshocks.

In [ ]:
# read the mainshock timing
fi_mainshocktiming = "../../../GougeEventStats/Mainshocktiming/data/MainshockTiming_FB03_087.csv"

df_mainshock = pd.read_csv(fi_mainshocktiming, index_col=0)
df_mainshock.head()

In [ ]:
df_gc_selected.head()

In [ ]:
for event_id, row in df_gc_selected.iterrows():

    if row.event_label == "A":
        t_hold = 0
    else:
        t0 = df_mainshock.loc[row["stickslip_id"]-1, "mainshock_termination_all"]
        t1 = row.event_onset_time
        t_hold = t1-t0

    df_gc_selected.loc[event_id, "t_hold"] = t_hold
    

In [ ]:
df_gc_selected.head()

In [ ]:
df_gc_selected.t_hold

In [ ]:
len(df_gc_fore)

In [ ]:
scatter_mc = sns.color_palette("Set1")
scatter_mc

In [ ]:
def M02Mw(M0):
    return (np.log10(M0) - 9.1) * 2.0 / 3.0 # synchronized with OpenSWPC : moment_magnitude ( m0 )

def Mw2M0(Mw):
    return 10**( 1.5 * Mw + 9.05) # synchronized with OpenSWPC : seismic_moment ( mw )
   

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5, 5.4))
mainmarkersize=7

df_gc_fore = df_gc_selected[df_gc_selected["event_label"]=="F"]
# df_gc_fore.plot.scatter(x="t_hold", y="Mw", s=60, ax=ax, color="r", marker="o", edgecolor="k", zorder=3, label="Foreshock")
ax.errorbar(df_gc_fore["M0"].values, df_gc_fore["t_hold"].values, xerr=df_gc_fore["M0_std"].values,
           capsize=0, fmt="o", markersize=mainmarkersize, color=scatter_mc[0], lw=1, markeredgecolor = "k", label="Foreshocks", zorder=3)


ax.set_xlabel("$M_0$ [Nm]")
# ax.set_ylabel("Macroscopic Δτ [MPa]")
ax.set_ylabel("Hold time [s]")

ax.set_xscale('log')
ax.set_yscale('log')

xlimit_scaling = [5e-3, 7]
ax.set_xlim(xlimit_scaling)
ax.set_ylim([1, 15])

# ax.set_yticks(np.arange(20, 60, 10))
# ax.set_yticklabels(np.arange(20, 60, 10))

# plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
# plt.gca().yaxis.set_minor_formatter(ticker.NullFormatter())

ax.legend(loc=3)


#-----------------------#
#--- plot double axis---#
#-----------------------#

# https://pythonmatplotlibtips.blogspot.com/2018/01/add-second-x-axis-below-first-x-axis-python-matplotlib-pyplot.html
ax2 = ax.twiny()
ax2.set_xlabel("$M_w$")
ax2.set_xlim([M02Mw(xlimit_scaling[0]), M02Mw(xlimit_scaling[1])]) # synchronize with the first axis

# major tick
newlabel = np.array([-7.5, -7, -6.5, -6.0,])
ax2.minorticks_on()
ax2.set_xticks(newlabel)

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, zorder=-1)
ax.set_axisbelow('True')

plt.tight_layout()

plt.savefig(figdir+f"/M0vsThold_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.png", format="png", dpi=200)
# plt.savefig(figdir+f"/M0vsThold_fb03-{expr_id:03d}_{gougepatch_id}_Q{Qinv_quart}.pdf")
